In [ ]:
using LinearAlgebra, QuadGK, Roots, FFTW, FastGaussQuadrature
using VlasovSolvers
import VlasovSolvers: advection!
import VlasovSolvers: samples, Particles, PIC_step!, ParticleMover
using Plots
using ProgressMeter

using LaTeXStrings

In [ ]:
struct RectangleRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function RectangleRule(len, start, stop)
        points = LinRange(start, stop, len+1)[1:end-1]
        s = step(points) 
        weights = [s for _ = 1:len]
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct TrapezoidalRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function TrapezoidalRule(len, start, stop)
        points = LinRange(start, stop, len)[1:end]
        s = step(points) 
        weights = [s for _ = 1:len]
        weights[1] /= 2
        weights[end] /= 2
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct SimpsonRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function SimpsonRule(len, start, stop)
        # make sure the number of points is uneven
        if len % 2 == 0
            len += 1
        end
        points = LinRange(start, stop, len)
        s = step(points) 
        weights = s/3 .* ones(len)
        weights[2:2:end-1] .*= 4
        weights[3:2:end-2] .*= 2
        new(len, start, stop, vec(points), weights, s)
    end
end

In [ ]:
struct GaussLegendreRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussLegendreRule(len, start, stop)
        points, weights = gausslegendre(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct GaussRadauRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussRadauRule(len, start, stop)
        points, weights = gaussradau(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct GaussLobattoRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Array{Float64}
    weights :: Array{Float64}

    function GaussLobattoRule(len, start, stop)
        points, weights = gausslobatto(len)
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        new(len, start, stop, points, weights)
    end
end

In [ ]:
struct KronrodRule
    len  :: Int64
    start :: Float64
    stop  :: Float64
    points :: Vector{Float64}
    weights :: Vector{Float64}
    step :: Float64

    function KronrodRule(len, start, stop)
        pts, w, _ = kronrod(len)
        weights = []
        points = []
        for i = 1:len
            push!(points, pts[i])
            push!(weights, w[i])
            push!(points, -pts[i])
            push!(weights, w[i])
        end
        push!(points, pts[end])
        push!(weights, w[end])
        
        points .+= 1
        points .*= (stop - start) / 2
        points .+= start
        weights .*= (stop - start) / 2
        len = 2*len + 1
        new(len, start, stop, points, weights)
end
    end

In [ ]:
struct rkn_order4
    a :: Array{Float64, 2}
    b̄ :: Vector{Float64}
    c :: Vector{Float64}
    b :: Vector{Float64}
    dt :: Float64
    fg ::       Array{Float64, 2}
    G ::        Array{Float64, 1}
    tmpcos ::   Array{Float64, 1}
    tmpsin ::   Array{Float64, 1}
    C :: Vector{Float64}
    S :: Vector{Float64}

    function rkn_order4(X, dt)
        # a, b̄, c, b correspond to the Butcher tableau
        a = [0.0        0.0       0.0; 
            (2-√3)/12   0.0           0.0; 
            0.0         √(3)/6      0.0]
        b̄ = [(5 - 3*√3)/24,     (3+√3)/12,  (1+√3)/24]
        c = [(3+√3)/6,          (3-√3)/6,   (3+√3)/6]
        b = [(3-2*√3)/12,       1/2,        (3+2*√3)/12]

        new(a .* dt^2, b̄ .* dt^2, c .* dt, b .* dt, dt,
            zeros(Float64, length(X), 3), 
            similar(X), 
            similar(X),
            similar(X), [0.0], [0.0])
    end
end

In [ ]:
#= ==================== symplectic_RKN_order4!
Advect (X, V) on a time step dt using symplectic Runge-Kutta-Nystrom method of order4 [FQ10, sect.7.3, p.327, scheme1].

The equation satisfied by X is d^2(X(t)) / dt^2 = C(t)\cos(X(t)) - S(t)sin(X(t)).

RKN method considers Ẋ = V as a variable, and updates both X and V.

Using a change of variable and the jacobian of the flow being equal to 1, it is enough to know the function f on some
initial fixed grid, independant of time, to compute C(t) and S(t).

Args:
- X: matrix of positions at time t_n
- V: matrix of velocities at time t_n
- F: values of initial condition at time t_0
- rkn: rkn_order_4 struct, storing butcher tableau and pre-allocated arrays.
- kx: 2π/L

Updates X, V in place, and returns coefficients C, S at current time. 
==================== =#
function symplectic_RKN_order4!(X, V, F, rkn, kx)  
    @views begin
        for s=1:3
            rkn.G .= X .+ V .* rkn.c[s] .+ rkn.a[s, 1] .* rkn.fg[:, 1] .+ rkn.a[s, 2] .* rkn.fg[:, 2] .+ rkn.a[s, 3] .* rkn.fg[:, 3]
            
            rkn.G .*= kx
     
            rkn.tmpcos .= cos.(rkn.G)
            rkn.tmpsin .= sin.(rkn.G)
            sum!(rkn.C, rkn.tmpcos .* F)
            sum!(rkn.S, rkn.tmpsin .* F)
            rkn.C ./= π
            rkn.S ./= π
            rkn.fg[:, s] .= (rkn.C[1] .* rkn.tmpsin .- rkn.S[1] .* rkn.tmpcos)
        end
    
        X .+= rkn.dt .* V .+ rkn.b̄[1] .* rkn.fg[:, 1] .+ rkn.b̄[2]  .* rkn.fg[:, 2] .+ rkn.b̄[3]  .* rkn.fg[:, 3]
        V .+= rkn.b[1] .* rkn.fg[:, 1] .+ rkn.b[2] .* rkn.fg[:, 2] .+ rkn.b[3] .* rkn.fg[:, 3]
        rkn.tmpcos .= cos.(X .* kx)
        rkn.tmpsin .= sin.(X .* kx)
        sum!(rkn.C, rkn.tmpcos .* F)
        sum!(rkn.S, rkn.tmpsin .* F)
    end
    return @views rkn.C[1], rkn.S[1]
end

In [ ]:
function strang_splitting!(X, V, F, kx, dt)  
    X .+= V .* dt/2
    C = sum(cos.(X .* kx) .* F)
    S = sum(sin.(X .* kx) .* F)
    dphidx = (-C.*sin.(X .* kx) + S.*cos.(X .* kx)) ./ π
    V .-= dt .* dphidx
    X .+= V .* dt/2
    return C, S
end

## SL Généralisée

In [ ]:
function generalized_SL(nsteps::Int64, dt::Float64, quadrulex, quadrulev, kx, f0::Function; plotting=false::Bool)

    C = 0.0
    S = 0.0

    newX = vec(repeat(quadrulex.points, 1, quadrulev.len))   # newX[i, j] is position at current time when starting from x0_i, v0_j
    newV = vec(repeat(quadrulev.points', quadrulex.len, 1))  # newV[i, j] is velocity at current time when starting from x0_i, v0_j
    
    E²_elec  = Array{Float64}(undef, nsteps)
    E²_tot   = Array{Float64}(undef, nsteps)

    rkn = rkn_order4(newX, dt)

    if plotting
        widthx = -(-)(extrema(quadrulex.points)...)
        widthv = -(-)(extrema(quadrulev.points)...)
        scale = 25
    end
    
    progression = ProgressMeter.Progress(nstep,desc="Loop in time: ", showspeed=true)
        
    f0vals = vec(f0.(quadrulex.points, quadrulev.points') .* quadrulex.weights .* quadrulev.weights')

    animation = @animate for istep=1:nsteps
        C, S = symplectic_RKN_order4!(newX, newV, f0vals, rkn, kx)
        # C, S = strang_splitting!(newX, newV, f0vals, kx, dt)
        
        E²_elec[istep] = (C^2 + S^2) * quadrulex.stop / (2*π^2)
        E²_tot[istep] = (E²_elec[istep] + sum(newV.^2 .* f0vals)) / 2 
        
        newX[findall(x -> x > quadrulex.stop,  newX)] .-= quadrulex.stop - quadrulex.start
        newX[findall(x -> x < quadrulex.start, newX)] .+= quadrulex.stop - quadrulex.start
 
        if plotting
            # histogram2d(vec(newX), vec(newV), weights=vec(f0vals), bins=(nx, nv).*3, normalize=true, xlabel="position", ylabel="velocity", fillcolor=:roma)
            markerscale = 20
            plot(newX, newV, f0vals, seriestype=:scatter, markersize=sqrt(widthx * widthv * markerscale^2  / (nx*nv) / π), camera=(0, 90), markerstrokecolor="white", markerstrokewidth=0, label="", zcolor=vec(f0vals), c=:rainbow,aspect_ratio=:equal, size=(widthx, widthv).* 30)
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end
        ProgressMeter.next!(progression)
     end when plotting

     return sqrt.(E²_elec), sqrt.(E²_tot), animation, newX, newV, f0vals
end

## SL classique

In [ ]:
"""
    hmf_poisson!(fᵗ    :: Array{Complex{Float64},2},
                 mesh1 :: OneDGrid,
                 mesh2 :: OneDGrid,
                 ex    :: Array{Float64})

    Compute the electric hamiltonian mean field from the
    transposed distribution function

"""
function hmf_poisson!(fᵗ::Array{Complex{Float64},2},
        mesh1::OneDGrid,
        mesh2::OneDGrid,
        ex::Array{Float64})

    n1 = mesh1.len
    rho = mesh2.step .* vec(sum(fᵗ, dims=1)) # ≈ ∫ f(t,x_i,v)dv, i=1, ..., n1
    kernel = zeros(Float64, n1)
    k = -(mesh1.stop - mesh1.start) / (2π)
    kernel[2]   =  k    # fourier mode  1
    kernel[end] = -k    # fourier mode -1
    ex .= real(ifft(fft(rho) .* 1im .* kernel))
end

function solve_SL!(nsteps, dt, f, mesh1, mesh2, kx; plotting=false::Bool)
    n1, n2 = size(f)
    fᵗ = zeros(Complex{Float64}, (n2,n1))
    transpose!(fᵗ, f)

    energy² = Array{Float64}(undef, nsteps)
    etot² = Array{Float64}(undef, nsteps)

    ex = zeros(Float64, n1)
    hmf_poisson!(fᵗ, mesh1, mesh2, ex)
    advection!(fᵗ, mesh2, ex, 0.5dt)

    progression = ProgressMeter.Progress(nsteps,desc="Loop in time: ", showspeed=true)
    
    animation = @animate for istep = 1:nsteps
        energy²[istep] = sum(ex.^2) * mesh1.step
        etot²[istep] = (energy²[istep] + sum(mesh2.points'.^2 .* real(f)) * mesh1.step * mesh2.step) / 2
    
        advection!(f, mesh1, mesh2.points, dt)
        transpose!(fᵗ, f)
        hmf_poisson!(fᵗ, mesh1, mesh2, ex)
        advection!(fᵗ, mesh2, ex, dt)
        transpose!(f, fᵗ) 
        
        if plotting
            plot(mesh1.points, mesh2.points, real(f)', size=(500, 500), st=:surface, camera=(0, 90))
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end
        
        ProgressMeter.next!(progression)
    end when plotting
    if plotting
        return sqrt.(energy²), sqrt.(etot²), ex, animation
    else
        return sqrt.(energy²), sqrt.(etot²), ex, nothing
    end
end

# PIC solver

In [ ]:
function solve_PIC!(nsteps, dt, particles, meshx, kx; plotting=false::Bool)
    potential = []
    energy²_elec_from_phi = []
    energy²_elec_from_proj= []
    energy²_hamil_elec = []
    L = meshx.stop
    np = particles.nbpart

    pmover = ParticleMover(particles, meshx, kx, 1)
    
    if plotting
        widthx = -(-)(extrema(quadrulex.points)...)
        widthv = -(-)(extrema(quadrulev.points)...)
        scale = 25
    end
    
    progression = ProgressMeter.Progress(nsteps,desc="Loop in time: ", showspeed=true)
    animation = @animate for istep = 1:nsteps # Loop over time
        if plotting
            scale = 25
            plot(vec(p.x), vec(p.v), vec(p.wei), seriestype=:scatter, markersize=sqrt(widthx * widthv * scale^2  / (nx*nv) / π), camera=(0, 90),markerstrokecolor="white", markerstrokewidth=0, label="", zcolor=vec(p.wei), c=:rainbow,aspect_ratio=:equal, size=(widthx, widthv).*scale)
            title!("Progression: $(round(Int64,100*progression.counter / progression.n))%")
        end

        e₁², e₂², e₃² = PIC_step!(p, pmover, dt)
        push!(energy²_hamil_elec, e₁²)
        push!(energy²_elec_from_phi, e₂²)
        push!(energy²_elec_from_proj, e₃²)
        
        ProgressMeter.next!(progression)
    end when plotting
    return sqrt.(energy²_hamil_elec), sqrt.(energy²_elec_from_phi), sqrt.(energy²_elec_from_proj), animation
end

# INPUTS

## Landau damping

In [ ]:
kx = 0.5
L = 2π / kx
ϵ = 0.01
μ = 0.0
β = 1.0
f(x,v) = (1 + ϵ * cos(kx*x)) * exp(-β * (v-μ)^2 / 2) / √(2π/β)
castest = "Landau damping\n(kx, ϵ, μ, β) = $((kx, ϵ, μ, β)))"

## Two-Stream Instability

### Première version

In [ ]:
kx = 0.2
L = 2π / kx
ϵ = 0.001
β₁ = 1
β₂ = 1
v0 = 2.4
μ₁ =  v0
μ₂ = -v0
f(x,v) = (1 + ϵ * cos(kx * x)) * (exp(-β₁*(v-μ₁)^2 / 2) + exp(-β₁*(v-μ₂)^2 / 2)) / (√(2π)) * 0.5
castest = "TSI\n(kx, β₁, β₂, μ₁, μ₂) = $((kx, β₁, β₂, μ₁, μ₂))"

### Seconde version

In [ ]:
kx = 0.2
L = 2π / kx
ϵ = 0.001
β = 1
μ =  0
f(x,v) = (1 + ϵ * cos(kx * x)) * v^2 * exp(-β*(v-μ)^2 / 2) / (√(2π))
castest = "TSI\n(kx, β, μ) = $((kx, β, μ))"

## Bump-on-Tail

In [ ]:
kx = 0.3
L = 2π / kx
ϵ = 0.04
β₁ = 1
β₂ = 4
μ₁ = 0
μ₂ = 4.5
nb = 0.2
np = 0.9
f(x,v) = (1 + ϵ * cos(kx*x)) * (np*exp(-β₁*(v-μ₁)^2 /2) + nb*exp(-β₂*(v-μ₂)^2 / 2)) / √(2π)
castest = "Bump on tail\n(kx, ϵ, β₁, β₂, μ₁, μ₂, nb, np) = $((kx, ϵ, β₁, β₂, μ₁, μ₂, nb, np))"

## Strong Landau Damping

In [ ]:
kx = 0.5
L = 2π / kx
ϵ = 0.5
β = 1.0
μ = 0.0
f(x,v) = (1 + ϵ * cos(kx*x)) * exp(-β * v^2 / 2) / √(2π/β)
castest = "Strong Landau damping\n(kx, ϵ, μ, β) = $((kx, ϵ, μ, β)))"

# Inputs indépendants du cas test

In [ ]:
dev = CPU()

nstep = 1000
dt = 0.125

nx = 32
nv = 256

vmax = 6.5

meshx = OneDGrid(dev, nx, 0, L);
meshv = OneDGrid(dev, nv, -vmax, vmax);

quadX = TrapezoidalRule
quadV = TrapezoidalRule

quadrulex = quadX(nx, 0, L);
quadrulev = quadV(nv, -vmax, vmax);

# Simulations

In [ ]:
gsl  = zeros(Complex{Float64}, (nx,nv));
@. gsl = f.(meshx.points, meshv.points');
@time E_elecsl, E_totsl, _, animationsl = solve_SL!(nstep, dt, gsl, meshx, meshv, kx; plotting=false);

In [ ]:
# Solution de reference : 
# meshx2 = OneDGrid(dev, 4 * nx, 0, L) 
# meshv2 = OneDGrid(dev, 4 * nv, -4, 4) 

# gsl2  = zeros(Complex{Float64}, (4 * nx, 4 * nv));
# @. gsl2 = f.(meshx2.points, meshv2.points');
# @time E_elecsl2, E_totsl2, _, animationsl2 = solve_SL!(nstep * 4, dt / 4, gsl2, meshx2, meshv2, kx; plotting=false);

In [ ]:
plotornot = false;
@time E_elec, E_tot, animation, X, V, F = generalized_SL(nstep, dt, quadrulex, quadrulev, kx, f, plotting=plotornot);

# PIC interpretation

In [ ]:
np = nx*nv
x0 = vec(repeat(quadrulex.points, 1, quadrulev.len)) 
v0 = vec(repeat(quadrulev.points', quadrulex.len, 1))
wei = vec(f.(quadrulex.points, quadrulev.points') .* quadrulex.weights .* quadrulev.weights')
p = Particles(x0, v0, wei, np);

In [ ]:
@time energy_hamil_elec_gsl, energy_elec_from_phi_gsl, energy_elec_from_proj_gsl, animation = solve_PIC!(nstep, dt, p, meshx, kx, plotting=false);

# Classical PIC

In [ ]:
np = Int64(1e5)
(x0, y0, wei) = samples(np, kx, ϵ, μ, β)
p = Particles(x0, y0, wei, np);
@time energy_hamil_elec, energy_elec_from_phi, energy_elec_from_proj, animation = solve_PIC!(nstep, dt, p, meshx, kx;plotting=false);

# Plots

In [ ]:
t = (1:nstep) .* dt

# energies
# plot(t, E_elec, label=L"E_{elec charac},\quad dt="*"$(dt)", legend=:outertopright, minorgrid=true, size=(400, 200).*2)
# plot!(t, E_elecsl, label=L"E_{elec, SL},\quad dt="*"$(dt)")
# plot!(t, energy_hamil_elec_gsl, label=L"E_{elec, PIC charac},\quad dt="*"$(dt)")
# plot!(t, energy_hamil_elec, label=L"E_{elec, PIC},\quad dt="*"$(dt)")
# plot!(t, E_tot, label=L"E_{tot}", scaley=:log10)

# log(Energies)
plot(t, log.(E_elec),       label=L"\log(E_{elec, carac}),\quad dt="*"$(dt)", legend=:outertopright, minorgrid=true, size=(800, 400))
plot!(t, log.(E_elecsl),    label=L"\log(E_{elec, SL}),\quad dt="*"$(dt)", ls=:dash, lw=1)
# plot!(t, log.(E_tot),       label=L"\log(E_{tot}),\quad dt="*"$(dt)")
plot!(t, log.(energy_hamil_elec), label=L"\log(E_{hamil, PIC}),\quad dt="*"$(dt)", ls=:dot)
plot!(t, log.(energy_elec_from_proj), label=L"\log(E_{hamil, PIC, proj}),\quad dt="*"$(dt)", ls=:dot)
plot!(t, log.(energy_hamil_elec_gsl), label=L"\log(E_{hamil, PIC carac}),\quad dt="*"$(dt)", ls=:dot, lw=4)
# plot!(t, log.(energy_elec_from_phi_gsl), label=L"\log(E_{hamil_{elec}, PIC carac}),\quad dt="*"$(dt)", ls=:dashdot, lw=3)
# plot!(t, log.(energy_elec_from_proj_gsl), label=L"\log(E_{hamil, PIC carac, proj}),\quad dt="*"$(dt)", ls=:dot)
# plot!(t, log.(energy_from_projection), label=L"\log(E_{pic, PIC}),\quad dt="*"$(dt)")
# plot!(t, log.(energy_elec_from_phi), label=L"\log(E_{from\,\Phi, PIC}),\quad dt="*"$(dt)")


# ============== #

# Landau damping (kx=0.5):
# plot!(x->-0.1533x - 5.6, label="Damping attendu (-0.1533)")
# E_th = abs.(4ϵ * 0.3677 .* exp.(−0.1533 .* t) .* cos.(1.4156.*t .−0.5326245)) * sqrt(L/2)
# plot!(t, log.(E_th),label="Energie theorique")
# Landau damping (kx=0.4):
# plot!(x->-0.0661x - 5.3, label="Damping attendu (-0.0661)")
# E_th = 0.002.*0.42466.*abs.(cos.(1.285.*t .-0.33577)).*exp.(-0.0661.*t) # expression du bouquin, pas correcte
# E_th = abs.(4*ϵ*0.424666*exp.(-0.0661 .* t) .* cos.(1.2850 .* t .- 0.3357725) * sqrt(L/2)) # issue des calculs du bouquin en calculant correctement √(∫sin(0.5x)^2dx)
# plot!(t, log.(E_th),label="Energie theorique", ls=:dashdotdot)

# TSI (k,v0) = (0.2, 1.3):
# plot!(t, -0.001t .- 4.2, label=L"y=0.001t - 5.0")
# TSI (k,v0) = (0.2, 2.4):
# plot!(t, 0.2258t .- 6.4, label=L"y=0.2258t - 8.4")
# TSI (k,v0) = (0.2, 3):
# plot!(t, 0.2845t .- 6.1, label=L"y=0.2845t - 8.2")

# Strong Landau damping
# plot!(t, -0.285473t .+ 1, label=L"y=-0.285473t + 1")
# plot!(t, 0.086671t .- 3.8, label=L"y=0.086671t - 3.8")

title!(castest * "\n($(quadX)($(nx)), $(quadV)($(nv)))")
xlabel!("t")

# Plot annotations

In [ ]:
p = 2π / 1.4156
vline!(vcat([4 + i*p for i=0:4], [49.5 + i*p for i=0:3]), label="")
# vline!([28.1 + i*p for i=-4:0], label="")

In [ ]:
plot!([49.5, 49.5+p], [-10, -10], arrow=arrow(:closed, :both), label="")
annotate!((51.5, -10.3, text(L"\frac{2\pi}{1.4156}", :top, 10)))

# Divers

In [ ]:
E = E_tot
plot(t, (E .- E[1]) ./ E[1])

# Comparison with VlasovSolver's Bump on tail

In [ ]:
using VlasovSolvers
using Plots
using LaTeXStrings

In [ ]:
dev = CPU()                  # device
stepper = StrangSplitting()  # timestepper
dt = 0.05                     # timestep
nsteps = 8000                # total number of time-steps

α   = 0.04
kx  = 0.3

n1, n2 = 32, 64
x1min, x1max = 0.0, 20π
x2min, x2max = -9., 9.

mesh1 = OneDGrid(dev, n1, x1min, x1max)
mesh2 = OneDGrid(dev, n2, x2min, x2max)

fbot = DistributionFunction( mesh1, mesh2 )


for (i,x) in enumerate(mesh1.points), (j,v) in enumerate(mesh2.points)
    fbot.values[i,j]  = (1.0+α*cos(kx*x)) / (10*sqrt(2π)) * (9*exp(-0.5*v^2)+2*exp(-2*(v-4.5)^2))
end


In [ ]:
prob = VlasovProblem(fbot, BSLSpline(5), dev)

sol = VlasovSolvers.solve!(prob, stepper, dt, nsteps );

In [ ]:
plot(dt .* (1:nsteps), exp.(sol).^2, size=(600, 300))

# Comparison of quadratures on Bump on Tail

In [ ]:
dev = CPU()

nstep = 2000
dt = 0.05

nx = 32
nv = 64

vmax = 9.

meshx = OneDGrid(dev, nx, 0, L);
meshv = OneDGrid(dev, nv, -vmax, vmax);
gsl  = zeros(Complex{Float64}, (nx,nv));


rules = [RectangleRule, TrapezoidalRule, GaussLegendreRule, GaussRadauRule, GaussLobattoRule, KronrodRule]
t = (1:nstep) .* dt


for quadX = rules, quadV = rules
    quadrulex = quadX(nx, 0, L);
    quadrulev = quadV(nv, -vmax, vmax);

    @. gsl = f.(meshx.points, meshv.points');
    E_elecsl, E_totsl, _, animationsl = solve_SL!(nstep, dt, gsl, meshx, meshv, kx; plotting=false);
    E_elec, E_tot, animation, X, V, F = generalized_SL(nstep, dt, quadrulex, quadrulev, kx, f, plotting=plotornot);

    plot(t, E_elec, label=L"E_{elec},\quad dt="*"$(dt)", legend=:outertopright, minorgrid=true, size=(400, 200).*2)
    plot!(t, E_elecsl, label=L"E_{elec, SL},\quad dt="*"$(dt)", legend=:outertopright, minorgrid=true)

    title!(castest * "\n($(quadX)($(nx)), $(quadV)($(nv)))")
    xlabel!("t")
    savefig("/Users/ylehenaf/Documents/latex/imgs/methode_carac/comparison_quadratures/$(quadX)_$(quadV).png")
end
